**Google Analytics Customer Revenue Prediction**

To predict - How much GStore's Revenue from each customers, using EDA and ML models.

Given in problem description, the 80/20 rule - 80% of revenue comes from only 20% of the customers.So our task is to predict the Gstore's revenue from those customers (using natural log of sum of all transactions per user).

In [1]:
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb

**Features** of the given dataset: 
* fullVisitorId- A unique identifier for each user of the Google Merchandise Store.
* channelGrouping - The channel via which the user came to the Store.
* date - The date on which the user visited the Store.
* device - The specifications for the device used to access the Store.
* geoNetwork - This section contains information about the geography of the user.
* sessionId - A unique identifier for this visit to the store.
* socialEngagementType - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
* totals - This section contains aggregate values across the session.
* trafficSource - This section contains information about the Traffic Source from which the session originated.
* visitId - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.
* visitNumber - The session number for this user. If this is the first session, then this is set to 1.
* visitStartTime - The timestamp (expressed as POSIX time).

**Note:** 
Some columns contain serialised JSON as strings which should be deserialised and converted to seperate columns.


In [2]:
def load_df(csv_path='./train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, #json.loads takes in a string and converts to dict or list object.
                     dtype={'fullVisitorId': 'str'}, #convert id to string
                     nrows=nrows)
    

    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])  #converts semi-structured json to flat table.
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Shape: {df.shape}")
    return df

In [3]:
train_df = load_df("../input/train.csv")
test_df = load_df("../input/test.csv")
#train_df.to_csv("train_sep_cols")
#test_df.to_csv("test_sep_cols")
#train_df = load_df("./train_sep_cols")
#test_df = load_df("./test_sep_cols")

Shape: (903653, 55)
Shape: (804684, 53)


In [4]:
train_df.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Izmir,not available in demo dataset,Asia,Turkey,not available in demo dataset,not available in demo dataset,(not set),ttnet.com.tr,not available in demo dataset,Izmir,Western Asia,1,1,1,1,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Oceania,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,not available in demo dataset,not available in demo dataset,Australasia,1,1,1,1,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Madrid,not available in demo dataset,Europe,Spain,not available in demo dataset,not available in demo dataset,(not set),unknown.unknown,not available in demo dataset,Community of Madrid,Southern Europe,1,1,1,1,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not a

Lets perform EDA:

Each row in the dataset is one visit to the store.

Now lets check whether the given dataset conforms 80/20 rule :

* First convert values of totals.transactionRevenue to float 
* Group values according to fullVisitorId ( i.e we are calculating revenue from each customer )
* Then consider only those customers who have revenue more than zero and find out the ratio. 

In [5]:
train_df["totals.transactionRevenue"] = train_df["totals.transactionRevenue"].astype('float')
grouped_revenue = train_df.groupby("fullVisitorId")["totals.transactionRevenue"].sum().reset_index()

non_zero_customers = (grouped_revenue["totals.transactionRevenue"]>0).sum()
print("Number of unique customers with non-zero revenue : ", non_zero_customers, "and the ratio is : ", non_zero_customers / grouped_revenue.shape[0])

Number of unique customers with non-zero revenue :  9996 and the ratio is :  0.013996726255903731


So the ratio of revenue generating customers to total number of customers is 1.3%. From the above analysis it is confirmed that only 1.3% of the customers bring in revenue to Gstore.

**Data pre-processing:**

Some columns have constant values and missing values. So, Lets examine that and drop those columns from feature set which would make our dataset more useful while training models.



In [6]:
# convert the 'date' column values to datetime object
import datetime
train_df['date'] = train_df['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
test_df['date'] = test_df['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))

In [7]:
consts = [c for c in train_df.columns if train_df[c].nunique(dropna=False)==1 ] #lets include nan values in the count(nunique())
consts

['socialEngagementType',
 'device.browserSize',
 'device.browserVersion',
 'device.flashVersion',
 'device.language',
 'device.mobileDeviceBranding',
 'device.mobileDeviceInfo',
 'device.mobileDeviceMarketingName',
 'device.mobileDeviceModel',
 'device.mobileInputSelector',
 'device.operatingSystemVersion',
 'device.screenColors',
 'device.screenResolution',
 'geoNetwork.cityId',
 'geoNetwork.latitude',
 'geoNetwork.longitude',
 'geoNetwork.networkLocation',
 'totals.visits',
 'trafficSource.adwordsClickInfo.criteriaParameters']

As we know that train set has 55 columns whereas test set has 53 ,lets examine which of those two can be target variable.

In [8]:
set(train_df.columns).difference(set(test_df.columns))

{'totals.transactionRevenue', 'trafficSource.campaignCode'}

As 'trafficSource.campaignCode' is an extra feature not in test set (other than target variable 'totals.transactionRevenue' ).Lets drop this also. Even sessionId can be removed as this is just a unique number for each visit.

So lets drop consts (columns with constant values),  'sessionId', 'trafficSource.campaignCode'.

In [9]:
cols_to_drop = consts + ['sessionId'] + ["trafficSource.campaignCode"]
train_df = train_df.drop(cols_to_drop, axis=1)
test_df = test_df.drop(cols_to_drop[:-1], axis=1)

* Fill in missing values to 0.
* Now, Identify categorical variables and convert to numbers i.e label encode them.
* Identify numeric variables and convert them to floats.

**Note**: Do not include IDs and dates to any of the above operations.

In [10]:
#train_df.head()
#train_df.info()
#train_df.describe()

train_df["totals.transactionRevenue"].fillna(0, inplace=True)
train_y = train_df["totals.transactionRevenue"].values

#identify categorical variables and label encode them.
categorical_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']

for col in categorical_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))


numeric_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in numeric_cols:
    train_df[col] = train_df[col].astype(float)
    test_df[col] = test_df[col].astype(float)


channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.isTrueDirect


As the training set contains data from August 1st 2016 to August 1st 2017. Then take cross-validation set as last three month's data which makes the ratio of train set to cross-val set  roughly 7.5 : 2.5 .(This is considering months but not no of examples in trainset , so might not be exactly 7.5 : 2.5).

In [11]:
# Split the train dataset into development and valid based on time 
dev_df = train_df[train_df['date']<=datetime.date(2017,5,31)]
val_df = train_df[train_df['date']>datetime.date(2017,5,31)]

dev_y = np.log1p(dev_df["totals.transactionRevenue"].values)
val_y = np.log1p(val_df["totals.transactionRevenue"].values)

dev_X = dev_df[categorical_cols + numeric_cols] 
val_X = val_df[categorical_cols + numeric_cols] 
test_X = test_df[categorical_cols + numeric_cols] 

**Choosing a model to train:**

As we have large dataset (with more columns >30 and rows > 10000)its better to use ensemble learning (gradient boosting models).

Ensemble is a collection of predictors which come together (e.g. mean of all predictions) to give a final prediction.Boosting is an ensemble technique in which the predictors are not made independently, but sequentially (can overfit if stopping criteria isnt chosen well).

The gist on how Gradient Boosting Models work is, 
* A base model is created and is used to make predictions on the whole dataset.
* Then residual is calculated and observations which are incorrectly predicted are given higher weights.
* The next sequential model tries to correct the errors from the previous model.
* Similarly, multiple models are created, correcting the errors of the previous model.
* The final model (strong learner) is the weighted mean of all the models (weak learners).

 e1= y - y_pred1<br> 
 y_pred2 = y_pred1 + e1_pred<br>
 e2 = y - y_pred2<br> 
 and so on.
 
XGBoost and LightGBM are effiecient gradient boosting models.<br> 
XGBoost is a regularised boosting model and hence reduces overfitting. It also implements parallel processing and is faster compared to other boosting algorithms.<br>

Light GBM is fast, distributed gradient boosting framework. Unlike other boosting algorithms it splits the trees leafwise and not level wise.Leaf wise splitting may lead to overfitting. This can be avoided by specifying tree-specific hyper parameters like max depth.It trains faster(on larger datasets) compared to other boosting algorithms like XGBoost.Light GBM grows tree vertically,It will choose the leaf with max delta loss to grow.  <br>
Hence we'll use LightGBM to train our model and predict revenue from test set.

Note : Light GBM is sensitive to overfitting and can easily overfit small data. So the challenge in using LGBM is hyper-parameter tuning.




In [12]:
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 30,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    return pred_test_y, model, pred_val_y

# Training the model #
pred_test, model, pred_val = run_lgb(dev_X, dev_y, val_X, val_y, test_X)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 1.69586
[200]	valid_0's rmse: 1.69126
[300]	valid_0's rmse: 1.69175
Early stopping, best iteration is:
[254]	valid_0's rmse: 1.69087


 Let us compute the evaluation metric on the validation data. 
 * Assign zero if predicted value is less than 0.
 * Prepare a validation dataframe. As predicted values are logarithmic values apply np.exp ( we'll convert this to log values again after grouping values acc to 'fullVisitorId' and calculating sum of revenue per user )
 * Apply log on sum for all the transactions per user (or apply sum first on grouped data and then apply log).
 * Calculate rms error.

In [13]:
from sklearn import metrics

pred_val[pred_val<0] = 0
val_pred_df = pd.DataFrame({"fullVisitorId":val_df["fullVisitorId"].values})
val_pred_df["transactionRevenue"] = val_df["totals.transactionRevenue"].values
val_pred_df["PredictedRevenue"] = np.expm1(pred_val) #exp(x) -1 can also be used but expm1 gives greater precision when converting log

val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
val_pred_df["transactionRevenue"] = np.log1p(val_pred_df["transactionRevenue"].values)
val_pred_df["PredictedRevenue"] =  np.log1p(val_pred_df["PredictedRevenue"].values)

#Now apply rms to find out error
print(np.sqrt(metrics.mean_squared_error(val_pred_df["transactionRevenue"].values, val_pred_df["PredictedRevenue"].values)))

1.709925685736863


Prepare submission.csv :

In [14]:
train_id = train_df["fullVisitorId"].values
test_id = test_df["fullVisitorId"].values   
submit_df = pd.DataFrame({"fullVisitorId":test_id})

#Repeat same steps as we did for cross-validation
pred_test[pred_test<0] = 0
submit_df["PredictedLogRevenue"] = np.expm1(pred_test)
submit_df = submit_df.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
submit_df["PredictedLogRevenue"] = np.log1p(submit_df["PredictedLogRevenue"])

submit_df.columns = ["fullVisitorId", "PredictedLogRevenue"]
print(submit_df.head())
submit_df.to_csv("submission.csv", index=False)

         fullVisitorId  PredictedLogRevenue
0  0000000259678714014             0.516011
1  0000049363351866189             0.000000
2  0000053049821714864             0.007906
3  0000059488412965267             0.048398
4  0000085840370633780             0.011108
